# Kubernetes

In [277]:
# WEBDAV at http://10.15.136.41:30080/

import ipywidgets as widgets
from IPython.display import clear_output
from IPython.display import Javascript, display
import uuid

style = {'description_width': '250px'}
layout = {'width': '500px'}

def addVisText(key='',value=''):
    text = widgets.Text(description = key,value = value, style=style, layout=layout)
    display(text)  
    return text
    
def addVisCheckbox(key='',value=False):
    text = widgets.Checkbox(description = key,value = value, style=style, layout=layout)
    display(text)   
    return text

def getPort(peername='', name='peer-gossip'):
    callopt= 'jsonpath=\"{.spec.ports[?(@.name==\'' + name + '\')].nodePort}\"'
    try:    
        callProcess  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'get','service',peername,'-o', callopt])
        return str(callProcess)[3:-2]
    except Exception as e:
        return str(e)

def setPeerPorts(peername):
    os.environ['GOSSIP_PORT'] = getPort(peername=peername, name='peer-gossip')
    os.environ['PEER_LISTEN_PORT'] = getPort(peername=peername, name='peer-listen')
    os.environ['PEER_CHAINCODE_PORT'] = getPort(peername=peername, name='peer-chaincode')
    
def createConfig(peername='',peer_listen_port='',peer_gossip_port=''):
    try:  
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'create','configmap','hl-fabric-peer-' + peername,
                                          '--from-literal=hostname=kubernetes.research.dev.seeburger.de',
                                          '--from-literal=org_name=' + peername,
                                          '--from-literal=CORE_PEER_ADDRESS=kubernetes.research.dev.seeburger.de:' + peer_listen_port,
                                          '--from-literal=CORE_PEER_GOSSIP_EXTERNALENDPOINT=kubernetes.research.dev.seeburger.de:' + peer_gossip_port,
                                          '--from-literal=CORE_PEER_LOCALMSPID=' + peername + 'MSP'])
        return str(output)
    except Exception as e:
        return str(e)  
    
#!/home/jovyan/work/usr/bin/kubectl create configmap hl-fabric-peer-$PEER_NAME \
#         --from-literal=hostname=kubernetes.research.dev.seeburger.de \
#         --from-literal=org_name=$PEER_NAME \
#         --from-literal=CORE_PEER_ADDRESS=kubernetes.research.dev.seeburger.de:$PEER_LISTEN_PORT \
#         --from-literal=CORE_PEER_GOSSIP_EXTERNALENDPOINT=kubernetes.research.dev.seeburger.de:$GOSSIP_PORT \
#         --from-literal=CORE_PEER_LOCALMSPID=${PEER_NAME}MSP
            
def getPod(peername):
    try:    
        callProcess  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'get','pod','-l','app=' + peername,'-o', 
                                                'jsonpath=\"{.items[0].metadata.name}\"'])
        return str(callProcess)[3:-2]
    except Exception as e:
        return str(e)     
    
def getPods():
    !/home/jovyan/work/usr/bin/kubectl get pods
        
def setPeerEnvironment(peername):
    os.environ['POD_NAME'] = getPod(peername)
    os.environ['ORDERER_IP'] = '10.15.128.97'
    os.environ['ORDERER_HOSTNAME'] = 'orderer.example.com'
    os.environ['CHANNEL_NAME'] = 'mychannel'  
    setPeerPorts(peername)
    
def deletePeer(peername):
    try:    
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'delete','configmap','hl-fabric-peer-' + peername])
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'delete','-f','/home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/target/' + peername + '/k8s-peer.yaml'])
        output  = subprocess.check_output([ '/home/jovyan/work/usr/bin/kubectl', 'delete', 'service', peername])
        return output
    except Exception as e:
        return str(e)
    
#!/home/jovyan/work/usr/bin/kubectl  exec --stdin --tty $POD_NAME  -c scray-peer-cli /bin/bash "echo 'export ORDERER_CA=/tmp/tlsca.example.com-cert.pem' >> /root/.bashrc"    

#!/home/jovyan/work/usr/bin/kubectl  exec -it $POD_NAME -c scray-peer-cli -- /bin/bash -c "echo 'export ORDERER_CA=/tmp/tlsca.example.com-cert.pem' >> /root/.bashrc"

def addEnvVariable(pod_name, key,value):
    try:  
        cmd = "echo \'export '" + key + '=' + value + "'\' >> /root/.bashrc"
        #cmd = "echo 1"
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'exec','-it', pod_name, '-c', 'scray-peer-cli','--', '/bin/bash','-c',cmd])
        return output
    except Exception as e:
        return str(e)
        
def installAndAproveChaincode():
    # Host where the chaincode is running
    os.environ['IP_CC_SERVICE']='10.14.128.38'
    #Host where the example network is running
    os.environ['IP_OF_EXAMPLE_NETWORK']='10.15.128.97'
    !/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK

In [ ]:
class kubernetespeer(peername='',listen_port='',gossip_port=''):
    def __init__(self, addr = '', crt = '', pem = ''):
        self.peername = peername
        self.listen_port = listen_port
        self.gossip_port = gossip_port
    def getCertificate(self):
        try:
            return str(subprocess.check_output(['cat', self.crt]))[2:-3]   
        except Exception as e:
            return str(e)

# Multiple version

In [142]:
import os
import json
import subprocess
import pandas as pd

def handle_submit(text):
    global peername
    peername = text.value
    os.environ['PEER_NAME'] = peername
    !cd /home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/ && /home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/configure-deployment.sh -n $PEER_NAME
    !/home/jovyan/work/usr/bin/kubectl apply -f /home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/target/$PEER_NAME/k8s-peer-service.yaml
    print('done')
    setPeerPorts(peername)
    res = createConfig(peername=peername,peer_listen_port=os.environ['PEER_LISTEN_PORT'],peer_gossip_port=os.environ['GOSSIP_PORT'])
    print(res)
    !/home/jovyan/work/usr/bin/kubectl apply -f /home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/target/$PEER_NAME/k8s-peer.yaml
    
peertext = addVisText(key='peer',value='')
peertext.on_submit(handle_submit)      

deployment.apps/maja2 unchanged


Text(value='', description='peer', layout=Layout(width='500px'), style=DescriptionStyle(description_width='250…

In [291]:
peername='bbc1'
print (peername,os.environ['PEER_NAME'])
pod_name = getPod(peername)
os.environ['POD_NAME'] = pod_name
listen_port=getPort(peername=peername, name='peer-listen')
pod_name,listen_port

bbc1 bbc5


('bbc1-8447cc4b78-zpq9j', '32437')

In [290]:
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli touch /root/.bashrc
addEnvVariable(pod_name, 'ORDERER_CA', '/tmp/tlsca.example.com-cert.pem')
addEnvVariable(pod_name, 'ORDERER_HOSTNAME', 'orderer.example.com')
addEnvVariable(pod_name, 'CHANNEL_NAME', 'mychannel')
addEnvVariable(pod_name, 'CORE_PEER_ADDRESS', 'peer0.kubernetes.research.dev.seeburger.de:' + listen_port)
addEnvVariable(pod_name, 'CORE_PEER_MSPCONFIGPATH', '/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/users/Admin@kubernetes.research.dev.seeburger.de/msp')
addEnvVariable(pod_name, 'SHARED_FS_HOST', '10.15.136.41:30080')
addEnvVariable(pod_name, 'SHARED_FS_USER', 'scray')
addEnvVariable(pod_name, 'SHARED_FS_PW', 'scray')

b''

In [145]:
setPeerEnvironment(peername)

#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty 'lara2-5d8bd488d6-kll5n' -c lara2 -- /bin/bash pwd

kubectl exec [POD] [COMMAND] is DEPRECATED and will be removed in a future version. Use kubectl kubectl exec [POD] -- [COMMAND] instead.


In [240]:
#!/home/jovyan/work/usr/bin/kubectl  exec --stdin --tty $POD_NAME  -c scray-peer-cli chmod 755  /root/.bashrc
#!/home/jovyan/work/usr/bin/kubectl  exec --stdin --tty $POD_NAME  -c maja3 echo 'export ORDERER_CA=/tmp/tlsca.example.com-cert.pem' >> /root/.bashrc
#!/home/jovyan/work/usr/bin/kubectl  exec --stdin --tty $POD_NAME -c maja3 -- echo 'export ORDERER_CA=/tmp/tlsca.example.com-cert.pem' >> /tmp/.bashrc
########## execute commands in container
#!/home/jovyan/work/usr/bin/kubectl  exec -it $POD_NAME -c maja3 -- /bin/sh -c "ls -l / && hostname"
#!/home/jovyan/work/usr/bin/kubectl  exec -it $POD_NAME -c scray-peer-cli -- /bin/bash -c "ls -l / && hostname"
!/home/jovyan/work/usr/bin/kubectl  exec -it $POD_NAME -c scray-peer-cli -- /bin/bash -c "echo 'export ORDERER_CA=/tmp/tlsca.example.com-cert.pem' >> /root/.bashrc"

In [22]:
def downloadToLocal(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):  
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/' + remote_path + '/' + local_name, '-o', local_path + '/' + local_name])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)     
                    
                    
downloadToLocal(local_path='./', local_name='org3_update_in_envelope.pb' ,remote_path='upload')

''

In [102]:
setPeerEnvironment('tanja2')
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli --  /bin/sh export ORDERER_CA="/tmp/tlsca.example.com-cert.pem"
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli --  ORDERER_HOSTNAME="orderer.example.com"
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli --  CHANNEL_NAME="mychannel"
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli --  CORE_PEER_ADDRESS="peer0.kubernetes.research.dev.seeburger.de:32222"
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli --  CORE_PEER_MSPCONFIGPATH="/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/users/Admin@kubernetes.research.dev.seeburger.de/msp"


!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- env | fgrep ORDERER_HOSTNAME

/bin/sh: can't open 'export': No such file or directory
command terminated with exit code 2


In [241]:
peername='maja2'
setPeerEnvironment(peername)
os.environ['ORDERER_HOSTNAME'] = 'orderer.example.com'
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- env | fgrep ORDERER
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- which export
print(getPod(peername),getPort(peername=peername, name='peer-listen'))

maja2-54db4fffd6-lqzbr 30781


In [ ]:
export ORDERER_CA=/tmp/tlsca.example.com-cert.pem
export ORDERER_HOSTNAME="orderer.example.com"
export CHANNEL_NAME="mychannel"
export CORE_PEER_ADDRESS="peer0.kubernetes.research.dev.seeburger.de:32190"
export CORE_PEER_MSPCONFIGPATH="/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/users/Admin@kubernetes.research.dev.seeburger.de/msp"
export SHARED_FS_HOST=10.15.136.41:30080
export SHARED_FS_USER=scray
export SHARED_FS_PW=scray

curl --user $SHARED_FS_USER:$SHARED_FS_PW http://$SHARED_FS_HOST/ca/tlsca.example.com-cert.pem > /tmp/tlsca.example.com-cert.pem
curl --user scray:scray http://10.15.136.41:30080/upload/org3_update_in_envelope.pb -o ./org3_update_in_envelope.pb
peer channel signconfigtx -f org3_update_in_envelope.pb
curl --user scray:scray -T ./org3_update_in_envelope.pb http://10.15.136.41:30080/upload/org3_update_in_envelope.pb
  

In [89]:
!ls -l ./*.pb

#sign update
#createEnvironmentForPeer(localmspid=mspid2, tlsrootcert=crt2, mspconfigpath=msp2, peeraddress=addr2)
#setPeerEnvironment('carla1')
setPeerEnvironment('tanja2')
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- curl --user scray:scray http://10.15.136.41:30080/upload/org3_update_in_envelope.pb -o ./org3_update_in_envelope.pb
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- peer channel signconfigtx -f org3_update_in_envelope.pb
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- curl --user scray:scray -T ./org3_update_in_envelope.pb http://10.15.136.41:30080/upload/org3_update_in_envelope.pb
  

-rw-r--r-- 1 jovyan users 11348 Dec 22 17:46 ./org3_update_in_envelope.pb
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9666  100  9666    0     0   524k      0 --:--:-- --:--:-- --:--:--  555k
2020-12-23 12:07:28.595 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized


Step 3

In [288]:
print(getPod(peername))
setPeerEnvironment(peername)    
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP  $ORDERER_HOSTNAME $CHANNEL_NAME

maja3-d79b8d8fd-t9z4t
fetch http://dl-cdn.alpinelinux.org/alpine/v3.12/main/x86_64/APKINDEX.tar.gz
fetch http://dl-cdn.alpinelinux.org/alpine/v3.12/community/x86_64/APKINDEX.tar.gz
(1/1) Installing curl (7.69.1-r3)
7  0%                                                                           87100% ##########################################################################8Executing busybox-1.31.1-r16.trigger
OK: 29 MiB in 28 packages
(1/17) Installing fstrm (0.6.0-r1)
7  0%                                                                           8(2/17) Installing libgcc (9.3.0-r2)
7  0%                                                                           8(3/17) Installing krb5-conf (1.0-r2)
7  1%                                                                           8(4/17) Installing libcom_err (1.45.6-r0)
7  1%                                                                           8(5/17) Installing keyutils-libs (1.6.1-r1)
7  1% #                        

# Install and aprove chaincode

In [115]:
setPeerEnvironment('tanja2')
installAndAproveChaincode()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   450  100   450    0     0   3308      0 --:--:-- --:--:-- --:--:--  3308
2020-12-23 17:07:53.909 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 001 Installed remotely: response:<status:200 payload:"\nJbasic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee\022\tbasic_1.0" > 
2020-12-23 17:07:53.909 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 002 Chaincode code package identifier: basic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee
2020-12-23 17:07:56.108 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [c35fbb455b3f8f341299c7689289e2f39d1d428a1ad6a058bbc01eb18957f794] committed with status (VALID) at peer0.kubernetes.research.dev.seeburger.de:32190


In [116]:
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- lifecycle chaincode querycommitted --channelID mychannel --name basic --output json

OCI runtime exec failed: exec failed: container_linux.go:349: starting container process caused "exec: \"lifecycle\": executable file not found in $PATH": unknown
command terminated with exit code 126


In [117]:
#setPeerEnvironment('klaus')
!echo  $POD_NAME $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK

# Host where the chaincode is running
os.environ['IP_CC_SERVICE']='10.14.128.38'
#Host where the example network is running
os.environ['IP_OF_EXAMPLE_NETWORK']='10.15.128.97'
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK

tanja2-768f8dd45d-f6z66 10.14.128.38 10.15.128.97
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   450  100   450    0     0   1956      0 --:--:-- --:--:-- --:--:--  1956
Error: chaincode install failed with status: 500 - failed to invoke backing implementation of 'InstallChaincode': chaincode already successfully installed
Error: proposal failed with status: 500 - failed to invoke backing implementation of 'ApproveChaincodeDefinitionForMyOrg': attempted to redefine uncommitted sequence (1) for namespace basic with unchanged content
command terminated with exit code 1


In [ ]:
!ping orderer.example.com 7050

# ------------------------------------------------------------------------

# Shell only

In [ ]:
#!echo $GOSSIP_PORT, $PEER_NAME
getPort(peername=peername, name='peer-listen')

In [ ]:
GOSSIP_PORT=$(kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-gossip')].nodePort}")
PEER_LISTEN_PORT=$(kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-listen')].nodePort}")
PEER_CHAINCODE_PORT=$(kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-chaincode')].nodePort}")


In [ ]:
!echo $(/home/jovyan/work/usr/bin/kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-gossip')].nodePort}")
!echo $(/home/jovyan/work/usr/bin/kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-listen')].nodePort}")
!echo $(/home/jovyan/work/usr/bin/kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-chaincode')].nodePort}")

In [ ]:
os.environ['GOSSIP_PORT'] = "32068"  
os.environ['PEER_LISTEN_PORT'] = "30251"
os.environ['PEER_CHAINCODE_PORT'] = "30052"
!echo $GOSSIP_PORT
#!echo $(/home/jovyan/work/usr/bin/kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-gossip')].nodePort}")

In [ ]:
!/home/jovyan/work/usr/bin/kubectl create configmap hl-fabric-peer-$PEER_NAME \
 --from-literal=hostname=kubernetes.research.dev.seeburger.de \
 --from-literal=org_name=$PEER_NAME \
 --from-literal=CORE_PEER_ADDRESS=kubernetes.research.dev.seeburger.de:$PEER_LISTEN_PORT \
 --from-literal=CORE_PEER_GOSSIP_EXTERNALENDPOINT=kubernetes.research.dev.seeburger.de:$GOSSIP_PORT \
 --from-literal=CORE_PEER_LOCALMSPID=${PEER_NAME}MSP

In [ ]:
!/home/jovyan/work/usr/bin/kubectl apply -f /home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/target/$PEER_NAME/k8s-peer.yaml

In [ ]:
POD_NAME=$(/home/jovyan/work/usr/bin/kubectl  get pod -l app=peer0-org1-scray-org -o jsonpath="{.items[0].metadata.name}")
kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP  $ORDERER_HOSTNAME $CHANNEL_NAME

In [ ]:
!echo $(/home/jovyan/work/usr/bin/kubectl  get pod -l app=$PEER_NAME -o jsonpath="{.items[0].metadata.name}")


In [ ]:
os.environ['POD_NAME'] = "tom-6647c77bbb-hjlkc"
os.environ['ORDERER_IP'] = '10.15.128.97'
os.environ['ORDERER_HOSTNAME'] = 'orderer.example.com'
os.environ['CHANNEL_NAME'] = 'mychannel'

!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP  $ORDERER_HOSTNAME $CHANNEL_NAME

# Delete old peer

In [ ]:
!/home/jovyan/work/usr/bin/kubectl delete configmap hl-fabric-peer-klaus
!/home/jovyan/work/usr/bin/kubectl delete -f /home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/target/klaus/k8s-peer.yaml
!/home/jovyan/work/usr/bin/kubectl delete service klaus

In [76]:
#deletePeer('carla1')

service "tom" deleted


In [245]:
!/home/jovyan/work/usr/bin/kubectl get configmap 
!/home/jovyan/work/usr/bin/kubectl get deployments 
#!/home/jovyan/work/usr/bin/kubectl get svc -o json
!/home/jovyan/work/usr/bin/kubectl get svc

NAME                     DATA   AGE
hl-fabric-peer           5      22d
hl-fabric-peer-bb1       5      7d3h
hl-fabric-peer-bbc1      5      2m29s
hl-fabric-peer-bbc2      5      101s
hl-fabric-peer-bbc3      5      92s
hl-fabric-peer-bbc4      5      85s
hl-fabric-peer-maja      5      21h
hl-fabric-peer-maja3     5      20h
hl-fabric-peer-maja4     5      39m
hl-fabric-peer-maja5     5      6m24s
hl-fabric-peer-peer-42   5      15d
hl-fabric-peer-rr        5      15d
hl-fabric-peer-rrf3      5      15d
hl-fabric-peer-tanja1    5      7d3h
hl-fabric-peer-tanja2    5      7d1h
hl-fabric-peer-tanja3    5      7d1h
hl-fabric-peer-tom       5      15d
special-config           2      28d
NAME                              READY   UP-TO-DATE   AVAILABLE   AGE
bb1                               1/1     1            1           7d3h
bbc1                              0/1     1            0           2m29s
bbc2                              0/1     1            0           101s
bbc3               

# Create new peer and 'apply'

In [ ]:
!/home/jovyan/work/usr/bin/kubectl create configmap hl-fabric-peer \
 --from-literal=hostname=kubernetes.research.dev.seeburger.de \
 --from-literal=org_name=OrgSally \ 
 --from-literal=CORE_PEER_ADDRESS=kubernetes.research.dev.seeburger.de:30003 \
 --from-literal=CORE_PEER_GOSSIP_EXTERNALENDPOINT=kubernetes.research.dev.seeburger.de:30001 \
 --from-literal=CORE_PEER_LOCALMSPID=OrgSallyMSP

!/home/jovyan/work/usr/bin/kubectl apply -f https://raw.githubusercontent.com/scray/scray/feature/k8s-peer/projects/invoice-hyperledger-fabric/containers/k8s-peer.yaml        

1. Der neue Peer wurde erzeugt. All relevant infos and cryptos for the application is uploaded to external sharepoint
2. Now the application needs to be signed by other peers

# Now the peer can join the network

In [ ]:
import os
import json
import subprocess
import pandas as pd

!/home/jovyan/work/usr/bin/kubectl get pods
os.environ['POD_NAME'] = 'peer0-org1-scray-org-5c4d69f556-snz5c'

In [ ]:
#!/home/jovyan/work/usr/bin/kubectl get pods | fgrep peer0
#!POD_NAME=$(/home/jovyan/work/usr/bin/kubectl get pod -l app=peer0-org1-scray-org | tail -n 1 | cut -d' ' -f1)
os.environ['ORDERER_IP'] = '10.15.128.97'
os.environ['ORDERER_HOSTNAME'] = 'orderer.example.com'
os.environ['CHANNEL_NAME'] = 'mychannel'

!echo $POD_NAME
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP $ORDERER_HOSTNAME $CHANNEL_NAME

In [ ]:
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty peer0-org1-scray-org-84ddc5757f-glbgn -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK

# Install and aprove chaincode

In [ ]:
# Host where the chaincode is running
os.environ['IP_CC_SERVICE']='10.14.128.38'
#Host where the example network is running
os.environ['IP_OF_EXAMPLE_NETWORK']='10.15.128.97'
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK

# TMP

In [118]:
print(getPod(peername),getPort(peername=peername, name='peer-listen'))

tanja2-768f8dd45d-f6z66 32190


In [120]:
export ORDERER_CA="/tmp/tlsca.example.com-cert.pem"
export ORDERER_HOSTNAME="orderer.example.com"
export CHANNEL_NAME="mychannel"
export CORE_PEER_ADDRESS="peer0.kubernetes.research.dev.seeburger.de:32190"
export CORE_PEER_MSPCONFIGPATH="/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/users/Admin@kubernetes.research.dev.seeburger.de/msp"

peer lifecycle chaincode querycommitted --channelID mychannel --name basic --output json
   

SyntaxError: invalid syntax (<ipython-input-120-ef0d7b67c757>, line 1)

In [123]:
def readAsset(asset='asset1'):
    callopt= '{\"function\":\"ReadAsset\",\"Args\":[' + '\"' + asset + '\"' + ']}'
    try:    
        callProcess  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- peer', 'chaincode','query', '-C', 'mychannel', '-n', 'basic', '-c', callopt])
        return json.loads(str(callProcess)[2:-3])
    except Exception as e:
        return str(e)
    
readAsset('asset1')    

"[Errno 2] No such file or directory: '/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- peer'"

In [122]:
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- peer lifecycle chaincode querycommitted --channelID mychannel --name basic --output json

Error: failed to retrieve endorser client for querycommitted: endorser client failed to connect to kubernetes.research.dev.seeburger.de:32190: failed to create new connection: context deadline exceeded
Usage:
  peer lifecycle chaincode querycommitted [flags]

Flags:
  -C, --channelID string               The channel on which this command should be executed
      --connectionProfile string       The fully qualified path to the connection profile that provides the necessary connection information for the network. Note: currently only supported for providing peer connection information
  -h, --help                           help for querycommitted
  -n, --name string                    Name of the chaincode
  -O, --output string                  The output format for query results. Default is human-readable plain-text. json is currently the only supported format.
      --peerAddresses stringArray      The addresses of the peers to connect to
      --tlsRootCertFiles stringArray   If TLS i

In [124]:
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- peer

Usage:
  peer [command]

Available Commands:
  chaincode   Operate a chaincode: install|instantiate|invoke|package|query|signpackage|upgrade|list.
  channel     Operate a channel: create|fetch|join|list|update|signconfigtx|getinfo.
  help        Help about any command
  lifecycle   Perform _lifecycle operations
  node        Operate a peer node: start|reset|rollback|pause|resume|rebuild-dbs|upgrade-dbs.
  version     Print fabric peer version.

Flags:
  -h, --help   help for peer

Use "peer [command] --help" for more information about a command.
